<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/LLM_query_passage_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    !pip install openai -q

    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"

    PỲTHON="python3.8"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import json
import glob

from common_tools import *

import numpy as np

import pickle

import time

from datetime import datetime

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

In [6]:
os.getcwd()

'/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR'

### Merge the corpus parts

In [7]:
if not os.path.exists("clueweb22-pt_10M_sample/clueweb22-pt_colbertx.tsv"):
    corpus = np.sort(glob.glob("clueweb22-pt_10M_sample/clueweb22-pt_colbertx_0*.tsv"))

    with open("clueweb22-pt_10M_sample/clueweb22-pt_colbertx.tsv", "w") as output_file:
        for i, f1 in enumerate(corpus):

            if i > 0:
                output_file.write("\n")

            with open(f1) as input_file:
                for line in input_file:
                    output_file.write(line)
else:
    print("Corpus already merged...")

Corpus already merged...


### Continue the regular processing

In [8]:
DOCUMENT_RETRIEVALS_FOLDER="retrieval_pipelines"

In [9]:
EVALUATION_QUERIES="validation_queries/queries_validation.tsv"

CLUEWEB_PT_CORPUS="clueweb22-pt_10M_sample/clueweb22-pt_colbertx.tsv"

COLBERTX_TO_CLUEWEB_ID_TRANSLATION="clueweb22-pt_10M_sample/clueweb22-pt_colbertx_to_clueweb_id.tsv"

### Read the queries and the whole corpus

In [10]:
validation_queries_df = pd.read_csv(EVALUATION_QUERIES, "\t", header=None, names=['query_id', 'query_text'])

<ipython-input-10-dd081b983431>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  validation_queries_df = pd.read_csv(EVALUATION_QUERIES, "\t", header=None, names=['query_id', 'query_text'])


In [11]:
clueweb_pt_corpus_df = pd.read_csv(CLUEWEB_PT_CORPUS, sep='\t', header=None, names=["colbertx_id", "doc_text"])

In [12]:
clueweb_pt_corpus_df.head(2)

,colbertx_id,doc_text
0,0,Quarto Casal ou PNE– Máx. de Hóspedes: 2– Tamanho da cama: 1 cama de casalVer quartoQuarto Família– Máx. de Hóspedes: 4– Tamanho da cama: 1 cama de casal e 2 de solteiroVer quartoQuarto Single– Máx. de Hóspedes: 1– Tamanho da cama: 1 cama de solteiroVer quartoQuarto Twin– Máx. de Hóspedes: 2– Tamanho da cama: 2 camas de solteiroVer quartoFaça sua reservaReserve Agora!16 3515-6400
1,1,"Bolha Puig BMW F850GS 2018/... TOURING PLUS35954x de R$ 249,75R$ 999,00ou R$ 949,05 via Boleto BancárioBolha Puig Kawasaki R-Racer Zx10 16/2098494x de R$ 249,75R$ 999,00ou R$ 949,05 via Boleto BancárioBolha Puig Bmw F750Gs 2018/... Touring - 376837684x de R$ 249,75R$ 999,00ou R$ 949,05 via Boleto BancárioCamiseta VR|46 VRMTS204801VRMTS2048014x de R$ 14,97R$ 149,90 R$ 59,90ou R$ 56,90 via Boleto Bancário60Bolha Puig Suzuki Gsx-R 750 08/1446294x de R$ 199,75R$ 799,00ou R$ 759,05 via Boleto BancárioBolha Puig Kawasaki ZX10 RR 2022 Racing205414x de R$ 249,75A partir de R$ 999,00ou R$ 949,05 via Boleto BancárioDestaquesBolha Puig Bmw S1000rr 2014/19 - 756475644x de R$ 199,75A partir de R$ 799,00ou R$ 759,05 via Boleto BancárioBolha Puig BMW F850GS 2018/... TOURING PLUS35954x de R$ 249,75R$ 999,00ou R$ 949,05 via Boleto BancárioBolha Puig Bmw S1000r - 704070404x de R$ 224,75R$ 899,00ou R$ 854,05 via Boleto BancárioBolha Puig BMW F850GS 2018/ TOURING37694x de R$ 199,75R$ 799,00ou R$ 759,05 via Boleto BancárioBolha Puig Suzuki Gsx-R 750 2015/...56054x de R$ 199,75R$ 799,00ou R$ 759,05 via Boleto BancárioCamiseta VR|46 VRMTS261901VRMTS261901V4x de R$ 14,97R$ 149,90 R$ 59,90ou R$ 56,90 via Boleto Bancário60"


In [13]:
doc_id_translation_df = pd.read_csv(COLBERTX_TO_CLUEWEB_ID_TRANSLATION, sep='\t', header=None, names=['colbertx_id', 'clueweb_id'])

### Read all the retrieved documents for each validation query, considering the available pipelines

In [24]:
retrieval_results = glob.glob(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "mt5_*"))
retrieval_results += glob.glob(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "bm25_*"))
retrieval_results += glob.glob(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "colbertx_*"))

In [26]:
doc_retrievals = {}

In [27]:
for retrieval in retrieval_results:
    pipeline = os.path.basename(retrieval).split("_")[0]

    print("Processing {} pipeline...".format(pipeline))

    if pipeline == 'bm25':
        doc_retrievals[pipeline] = pd.read_csv(retrieval, sep='\t', header=None, names=['query_id', 'Q0', 'clueweb_id', 'bm25_position', 'bm25_score', 'bm25_hit', 'BM25'])

    elif pipeline == 'colbertx':
        retrievals_df = pd.read_csv(retrieval, sep='\t').sort_values(['query_id', 'score_znorm'], ascending=[True, False])



        doc_retrievals[pipeline] = retrievals_df.merge(doc_id_translation_df)

    else:
        mt5_pipeline_results = json.load(open(retrieval))

        mt5_scores = []

        for query_id, mt5_rerank in mt5_pipeline_results['bm25_mt5'].items():
            for doc_id, doc_stats in mt5_rerank.items():
                mt5_scores.append({'query_id': int(query_id),
                                   'clueweb_id': doc_id,
                                   'mt5_score': doc_stats['scores']})

        doc_retrievals[pipeline] = pd.DataFrame(mt5_scores)

Processing mt5 pipeline...
Processing bm25 pipeline...
Processing colbertx pipeline...


In [19]:
with open(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "doc_retrievals_all.pkl"), "wb") as output_file:
    pickle.dump(doc_retrievals, output_file, pickle.HIGHEST_PROTOCOL)

In [18]:
doc_retrievals.keys()

dict_keys(['mt5', 'bm25', 'colbertx'])

In [ ]:
doc_retrievals['bm25'].head(20)

In [ ]:
doc_retrievals['colbertx'].head(20)

In [ ]:
doc_retrievals['mt5']

### Now, work with LLM to validate the queries/passages association

In [ ]:
HOW_MANY_DOCUMENTS=1
MODEL_TO_USE=MODEL_GPT4

In [ ]:
if MODEL_TO_USE == MODEL_GPT4:
    initialize_openai("OPENAI_API_KEY_2")
else:
    initialize_openai()

In [ ]:
LLM_evaluations = {}

LLM_EVALUATION_KEY_FORMAT = "{:02}_{}"

In [ ]:
for pipeline, retrieved_docs in doc_retrievals.items():

    print("Handling {} pipeline...".format(pipeline))

    partial_merge_df = retrieved_docs.groupby('query_id').head(HOW_MANY_DOCUMENTS).reset_index(drop=True).merge(validation_queries_df)

    print(partial_merge_df.shape)

    if pipeline == 'colbertx':
        docs_to_evaluate_df = partial_merge_df.merge(clueweb_pt_corpus_df)

        score_original_field = "score_znorm"
    else:
        docs_to_evaluate_df = partial_merge_df.merge(doc_id_translation_df).merge(clueweb_pt_corpus_df)

        score_original_field = "{}_score".format(pipeline)

    for i, document in docs_to_evaluate_df.iterrows():

        document_key = LLM_EVALUATION_KEY_FORMAT.format(document['query_id'], document['clueweb_id'])

        print("Query/Passage evaluation {}; document_key={}...".format(i, document_key))

        if document_key not in LLM_evaluations:
            try:
                relevance_results = execute_LLM_passage_relevance_evaluation(document['query_text'], document['doc_text'], model=MODEL_TO_USE, verbose=False)
            except Exception as e:
                print(e)

                time.sleep(10)

                relevance_results = execute_LLM_passage_relevance_evaluation(document['query_text'], document['doc_text'], model=MODEL_TO_USE, verbose=False)

            relevance_results['query_text'] = document['query_text']
            relevance_results['colbertx_id'] = document['colbertx_id']
            relevance_results['doc_text'] = document['doc_text']
        else:
            print("-- LLM already evaluated document {}...\n".format(document_key))

            relevance_results = LLM_evaluations[document_key]

        relevance_results['{}_score'.format(pipeline)] = document[score_original_field]

        LLM_evaluations[document_key] = relevance_results

        if (i > 0) and (i % 10 == 0):
            with open(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "LLM_top_{:03}_validation_queries_evaluation_tmp.pkl".format(HOW_MANY_DOCUMENTS)), "wb") as output_file:
                pickle.dump(LLM_evaluations, output_file, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "LLM_top_{:03}_validation_queries_evaluation_tmp.pkl".format(HOW_MANY_DOCUMENTS)), "wb") as output_file:
    pickle.dump(LLM_evaluations, output_file, pickle.HIGHEST_PROTOCOL)

Handling mt5 pipeline...
(100, 4)
Query/Passage evaluation 0; document_key=00_clueweb22-pt0000-55-02859_2...

LLM query relevance evaluation duration: 4.419157981872559; cost: 0.028229999999999998


Query/Passage evaluation 1; document_key=01_clueweb22-pt0001-22-15255_0...

LLM query relevance evaluation duration: 6.5096893310546875; cost: 0.02937


Query/Passage evaluation 2; document_key=02_clueweb22-pt0001-73-10674_5...

LLM query relevance evaluation duration: 4.429004907608032; cost: 0.02898


Query/Passage evaluation 3; document_key=03_clueweb22-pt0001-51-10267_21...

LLM query relevance evaluation duration: 4.663001537322998; cost: 0.02664


Query/Passage evaluation 4; document_key=04_clueweb22-pt0000-74-01583_6...

LLM query relevance evaluation duration: 5.754964590072632; cost: 0.02712


Query/Passage evaluation 5; document_key=05_clueweb22-pt0001-43-06825_19...

LLM query relevance evaluation duration: 4.568722248077393; cost: 0.02625


Query/Passage evaluation 6; document_k

In [ ]:
len(LLM_evaluations)

292

In [ ]:
validation_results_df = pd.DataFrame.from_dict(LLM_evaluations, orient='index')

In [ ]:
validation_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
validation_results_df.to_csv(os.path.join(DOCUMENT_RETRIEVALS_FOLDER, "LLM_top_{:03}_{}_validation_queries_evaluation_{}.tsv".format(HOW_MANY_DOCUMENTS, MODEL_TO_USE, validation_timestamp)),
                             sep='\t')